# Exercise #1: PRMS for entity ranking

## Build a local Elasticsearch index of a set of selected movies

Build a fielded Elasticsearch index. Fields should include title, description, categories, directors, actors.

In [1]:
# TODO

## MLM for ranking movies

Implement the mixture of language models for ranking movies

Step 1: perform initial retrieval

In [2]:
# TODO

Step 2: re-ranking using MLM

In [3]:
# TODO

## PRMS

Implement field-specific term weighting using PRMS

In [4]:
# TODO